# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow

tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-macos==2.10.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2023-11-21 20:44:05.561760: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJJ3W6D/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteJJ3W6D/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2023-11-21 20:44:32.227763: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

In [7]:
wv['dog']

array([-0.17309615,  0.15758842, -0.23145853,  0.23177603, -0.02615073,
       -0.36018732, -0.00635529,  0.6371121 , -0.20603655, -0.22077453,
       -0.01554342, -0.34829304, -0.03583476,  0.17890297, -0.04930957,
       -0.3005334 ,  0.21870801, -0.3013717 ,  0.01972173, -0.36416826,
        0.24191941,  0.09235658,  0.33126524, -0.14503357, -0.03104501,
        0.01940642, -0.29300994, -0.07446988, -0.17698368,  0.0425458 ,
        0.22310834, -0.00105166,  0.14976552, -0.42061538, -0.17121936,
        0.19314305,  0.17524293, -0.19813286, -0.26038358, -0.30745137,
       -0.1414239 , -0.28940076, -0.25525698,  0.13858297,  0.26230726,
       -0.02540751, -0.1545074 , -0.08063949,  0.219734  ,  0.14521518,
        0.06682689, -0.18002981, -0.18439114, -0.04813604, -0.18781628,
        0.16295165,  0.20566793, -0.07419734, -0.19818065,  0.04436482,
        0.1032194 ,  0.00461303,  0.00162991,  0.03852154, -0.1805421 ,
        0.15668152, -0.03643076,  0.18483333, -0.25378722,  0.26

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [11]:
# YOUR CODE HERE
wv['hello']

array([-0.01470163,  0.06708803, -0.09059665,  0.09554624, -0.03969054,
       -0.16038364,  0.05447185,  0.26675937, -0.12261415, -0.13548523,
        0.02034144, -0.16055408, -0.03027936,  0.09957619, -0.04282329,
       -0.12441508,  0.12225334, -0.08715156, -0.00956588, -0.23976289,
        0.09011167,  0.02802246,  0.15667208, -0.05721239, -0.04251176,
        0.03566956, -0.07910535, -0.05418784, -0.09202748,  0.01218196,
        0.10960547,  0.03161328,  0.03812117, -0.18834847, -0.11893881,
        0.09001625,  0.08739992, -0.08175941, -0.13937241, -0.14924654,
       -0.04152436, -0.12631476, -0.07243185,  0.03977467,  0.13380606,
        0.00196732, -0.08113959, -0.07728238,  0.11442323,  0.0896459 ,
        0.0096111 , -0.11507533, -0.10456835, -0.03123387, -0.12434633,
        0.08083215,  0.1127295 , -0.063553  , -0.0691224 , -0.00459452,
        0.0784948 , -0.03484884,  0.02578934,  0.04014636, -0.05449207,
        0.10202088, -0.04031819,  0.06877428, -0.10858061,  0.07

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [12]:
# YOUR CODE HERE
len(wv)

8006

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [19]:
# YOUR CODE HERE
wv.most_similar('dog')

[('bill', 0.9901387691497803),
 ('passion', 0.9899961948394775),
 ('birth', 0.9879319071769714),
 ('calls', 0.9878493547439575),
 ('costume', 0.9878250360488892),
 ('wear', 0.987511932849884),
 ('beetle', 0.9869688749313354),
 ('rush', 0.9869322776794434),
 ('killing', 0.9866613745689392),
 ('ugly', 0.9866102337837219)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [17]:
# YOUR CODE HERE
wv.similar_by_vector('dog')

[('bill', 0.9901387691497803),
 ('passion', 0.9899961948394775),
 ('birth', 0.9879319071769714),
 ('calls', 0.9878493547439575),
 ('costume', 0.9878250360488892),
 ('wear', 0.987511932849884),
 ('beetle', 0.9869688749313354),
 ('rush', 0.9869322776794434),
 ('killing', 0.9866613745689392),
 ('ugly', 0.9866102337837219)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [20]:
# YOUR CODE HERE
wv['good']-wv['bad']

array([-0.03534961,  0.13502872,  0.4137385 ,  0.38134372,  0.08823401,
       -0.8201192 , -0.34709102,  0.5792955 , -0.4194067 , -0.5997101 ,
        0.10579137, -0.3826105 ,  0.09660155,  0.03918004,  0.0196915 ,
        0.17720737, -0.2612007 ,  0.27284008, -0.28088847, -0.19402492,
       -0.0815537 ,  0.19209409, -0.06904507,  0.03449383,  0.05863795,
       -0.09741513, -0.18374795,  0.46335515, -0.22026603, -0.61876047,
       -0.40651238,  0.08033217,  0.3751334 , -0.01497418, -0.19014846,
       -0.14551592,  0.5160606 , -0.23707592, -0.02483051,  0.776091  ,
       -0.01726694, -0.15126985, -0.41234154, -0.09473699,  0.23787297,
       -0.214164  ,  0.75478876, -0.00763762, -0.5257641 ,  0.11956757,
       -0.15187511,  0.1697348 , -0.37857434, -0.05014628,  0.10503468,
        0.11581016, -0.07891062,  0.12222585,  0.1952622 ,  0.39292192,
        0.1188639 ,  0.50199234, -0.2640587 , -0.48301113,  0.3211748 ,
       -0.09945762, -0.10453796, -0.21628675,  0.6184786 ,  0.34

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [22]:
# YOUR CODE HERE
res = wv['good']-wv['bad']+wv['stupid']

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [23]:
# YOUR CODE HERE
wv.similar_by_vector(res)

[('nice', 0.7718669176101685),
 ('good', 0.7377179265022278),
 ('great', 0.7305476069450378),
 ('also', 0.7291788458824158),
 ('decent', 0.7122593522071838),
 ('deed', 0.6961852312088013),
 ('baldwin', 0.6907151937484741),
 ('biased', 0.6905772686004639),
 ('done', 0.6896340847015381),
 ('scary', 0.6862273216247559)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [24]:
res = wv['boy']-wv['girl']+wv['women']

In [25]:
wv.similar_by_vector(res)

[('shorts', 0.9538692831993103),
 ('days', 0.950776994228363),
 ('kinds', 0.9500086307525635),
 ('versions', 0.9410078525543213),
 ('types', 0.9284066557884216),
 ('episodes', 0.9281140565872192),
 ('names', 0.9230408072471619),
 ('countries', 0.9213932752609253),
 ('christians', 0.92014479637146),
 ('share', 0.9137395024299622)]

In [26]:
# YOUR CODE HERE
res = wv['queen']-wv['king']+wv['actor']
wv.similar_by_vector(res)

[('actor', 0.9772409200668335),
 ('actress', 0.8732454776763916),
 ('role', 0.8584352135658264),
 ('guy', 0.8497487306594849),
 ('pet', 0.837003231048584),
 ('performance', 0.8369792699813843),
 ('job', 0.8066487312316895),
 ('informs', 0.8016856908798218),
 ('man', 0.7943729758262634),
 ('inexperienced', 0.7785251140594482)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [31]:
# YOUR CODE HERE
wv = Word2Vec(sentences=X_train,vector_size=200)

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [39]:
# YOUR CODE HERE
len(wv.wv.key_to_index),len(X_train)

(8006, 2500)

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [40]:
# YOUR CODE HERE
wv_3 = Word2Vec(sentences=X_train,vector_size=200,min_count=7)
wv_4 = Word2Vec(sentences=X_train,vector_size=200,min_count=3)

In [42]:
len(wv_3.wv.key_to_index),len(wv_4.wv.key_to_index)

(6064, 12116)

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [43]:
# YOUR CODE HERE
wv_5 = Word2Vec(sentences=X_train,vector_size=200,min_count=4,window=6)

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [61]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec:Word2Vec, sentence):
    matrix_list = []
    for word in sentence:
        if word in word2vec.wv:
            matrix_list.append(word2vec.wv[word])
    return np.array(matrix_list)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [62]:
def embedding(word2vec, sentences):
    embedded_list = []
    for sentence in sentences:
        embedded_list.append(embed_sentence(word2vec,sentence))
    return embedded_list
    
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [ ]:
from tensorflow.keras.

In [ ]:
X_train_pad = pad

In [63]:
### YOUR CODE HERE

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)

NameError: name 'X_train_pad' is not defined



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
